In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.utils.tensorboard import SummaryWriter
import os
import matplotlib.pyplot as plt
import numpy as np

# Hyperparameters
latent_dim = 20
input_dim = 784  # MNIST 28x28
hidden_dim = 400
num_samples = 5  # Number of importance samples
batch_size = 128
epochs = 10
learning_rate = 1e-3

# Create directories for saving outputs
os.makedirs('./results/vae_samples', exist_ok=True)
os.makedirs('./logs', exist_ok=True)

# Initialize TensorBoard writer
writer = SummaryWriter('./logs/vae_importance_sampling')

# VAE Model (same as before)
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        
        # Encoder
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, latent_dim)  # mu
        self.fc22 = nn.Linear(hidden_dim, latent_dim)  # logvar
        
        # Decoder
        self.fc3 = nn.Linear(latent_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, input_dim)
    
    def encode(self, x):
        h = F.relu(self.fc1(x))
        return self.fc21(h), self.fc22(h)
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def decode(self, z):
        h = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h))
    
    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, input_dim))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

# Importance-weighted loss function with logging
def importance_weighted_loss(recon_x, x, mu, logvar, num_samples=5, global_step=0):
    batch_size = x.size(0)
    x = x.view(-1, input_dim)
    
    # Repeat data for multiple samples
    x = x.repeat(num_samples, 1)
    mu = mu.repeat(num_samples, 1)
    logvar = logvar.repeat(num_samples, 1)
    
    # Sample z using reparameterization
    std = torch.exp(0.5 * logvar)
    eps = torch.randn_like(std)
    z = mu + eps * std
    
    # Decode samples
    recon_x = torch.sigmoid(model.fc4(F.relu(model.fc3(z))))
    
    # Calculate log probabilities
    log_p_x_given_z = -F.binary_cross_entropy(recon_x, x, reduction='none').sum(1)
    log_p_z = Normal(0, 1).log_prob(z).sum(1)
    log_q_z_given_x = Normal(mu, std).log_prob(z).sum(1)
    
    # Calculate log weights
    log_w = log_p_x_given_z + log_p_z - log_q_z_given_x
    
    # Reshape to (num_samples, batch_size) and compute log mean exp
    log_w = log_w.view(num_samples, batch_size)
    log_likelihood = torch.logsumexp(log_w, dim=0) - torch.log(torch.tensor(num_samples, dtype=torch.float32))
    
    # Log statistics to TensorBoard
    writer.add_scalar('Loss/log_likelihood', log_likelihood.mean(), global_step)
    writer.add_scalar('Loss/KL_divergence', (log_q_z_given_x.mean() - log_p_z.mean()), global_step)
    writer.add_histogram('Latent/mu', mu, global_step)
    writer.add_histogram('Latent/logvar', logvar, global_step)
    
    # Return negative log likelihood (to be minimized)
    return -log_likelihood.mean()

# Visualization functions
def visualize_reconstruction(data, epoch):
    with torch.no_grad():
        sample = data[:8]
        recon_batch, _, _ = model(sample)
        
        comparison = torch.cat([sample.view(8, 1, 28, 28), 
                               recon_batch.view(8, 1, 28, 28)])
        save_image(comparison.cpu(),
                 f'./results/vae_samples/reconstruction_{epoch}.png', nrow=8)
        
        # Add to TensorBoard
        writer.add_images('Reconstruction/original_vs_reconstructed', 
                         comparison.unsqueeze(1), epoch)

def visualize_latent_space(data_loader, epoch):
    with torch.no_grad():
        latent_vectors = []
        labels = []
        for data, label in data_loader:
            mu, _ = model.encode(data.view(-1, input_dim))
            latent_vectors.append(mu)
            labels.append(label)
        
        latent_vectors = torch.cat(latent_vectors, dim=0).cpu().numpy()
        labels = torch.cat(labels, dim=0).cpu().numpy()
        
        # Plot 2D visualization if latent_dim >= 2
        if latent_dim >= 2:
            plt.figure(figsize=(10, 10))
            plt.scatter(latent_vectors[:, 0], latent_vectors[:, 1], c=labels, 
                      cmap='tab10', alpha=0.5)
            plt.colorbar()
            plt.title(f'Latent Space Visualization (Epoch {epoch})')
            plt.savefig(f'./results/vae_samples/latent_space_{epoch}.png')
            plt.close()
            
            # Add to TensorBoard
            writer.add_figure('Latent/2D_visualization', plt.gcf(), epoch)

def generate_random_samples(epoch, num_samples=8):
    with torch.no_grad():
        # Sample from prior
        z = torch.randn(num_samples, latent_dim)
        samples = model.decode(z)
        
        # Save generated samples
        save_image(samples.view(num_samples, 1, 28, 28),
                 f'./results/vae_samples/generated_{epoch}.png', nrow=4)
        
        # Add to TensorBoard
        writer.add_images('Generation/random_samples', 
                         samples.view(-1, 1, 28, 28), epoch)

# Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Test dataset for latent space visualization
test_dataset = datasets.MNIST('./data', train=False, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=True)

# Initialize model and optimizer
model = VAE()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop with logging and visualization
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        global_step = (epoch - 1) * len(train_loader) + batch_idx
        optimizer.zero_grad()
        
        # Forward pass
        mu, logvar = model.encode(data.view(-1, input_dim))
        
        # Calculate importance-weighted loss
        loss = importance_weighted_loss(None, data, mu, logvar, 
                                       num_samples=num_samples, 
                                       global_step=global_step)
        
        # Backward pass
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item() / len(data):.6f}')
            
            # Log gradients and parameters
            for name, param in model.named_parameters():
                writer.add_histogram(f'Params/{name}', param, global_step)
                if param.grad is not None:
                    writer.add_histogram(f'Grads/{name}', param.grad, global_step)
    
    avg_loss = train_loss / len(train_loader.dataset)
    print(f'====> Epoch: {epoch} Average loss: {avg_loss:.4f}')
    writer.add_scalar('Loss/train', avg_loss, epoch)
    
    # Visualization at the end of each epoch
    with torch.no_grad():
        # Get a batch of test data for visualization
        test_data, _ = next(iter(test_loader))
        
        # Visualizations
        visualize_reconstruction(test_data, epoch)
        visualize_latent_space(test_loader, epoch)
        generate_random_samples(epoch)

# Run training
for epoch in range(1, epochs + 1):
    train(epoch)

# Close TensorBoard writer
writer.close()